In [4]:
import email, getpass, imaplib, os, re
import justext
from tqdm import tqdm

In [5]:
user = input("Enter your GMail username --> ")
pwd=getpass.getpass()

Enter your GMail username --> rahejakartikay99@gmail.com
········


In [6]:
#BEFORE USING, IMAP ENABLE IMAP AND POP ACCESS OF GMAIL ACCOUNT IN GMAIL SETTINGS

In [7]:
m = imaplib.IMAP4_SSL("imap.gmail.com")
m.login(user,pwd)
m.select('inbox')

typ,data=m.search(None,'TEXT "Thank you for applying"')
print('IDs of mails containing parts of substring Thank you for applying')
data


IDs of mails containing parts of substring Thank you for applying


[b'5197 6091 6355 8166 10942 11582 12777 12972 13346 16678 16689 17605 18813 19025 19414 19424 20290 20375 20476 20628 20674 20977 21018 21241 21357 21914 23152 23700 24373 24374 24448 24526 24531 24601 24604 24851 24876 25636 25777 25780 25887 26040 26042 26046 26127']

In [8]:
ids = data[0] # data is a list.
id_list = ids.split() # ids is a space separated string

In [9]:
#Fetching email boies of selected emails
mails=[]
for i in id_list:
    result, mail = m.fetch(i, "RFC822") # fetch the email body (RFC822) for the given ID
    mails.append(mail)

In [10]:
mails[1][0][1]
#Fetched mail contains a lot of boilerplate content so this has to be cleaned

b'Delivered-To: rahejakartikay99@gmail.com\r\nReceived: by 10.223.164.74 with SMTP id e10csp1160042wra;\r\n        Tue, 20 Jun 2017 08:04:42 -0700 (PDT)\r\nX-Received: by 10.99.101.135 with SMTP id z129mr31773985pgb.66.1497971081881;\r\n        Tue, 20 Jun 2017 08:04:41 -0700 (PDT)\r\nARC-Seal: i=1; a=rsa-sha256; t=1497971081; cv=none;\r\n        d=google.com; s=arc-20160816;\r\n        b=PxExouW/fPqQ9mKsAw1bTVMEMTw3fSH9cJdvktzdc/FWMS5hrZr0Yh9OhWzWbiWS5i\r\n         VKdPDx8/TDDL1Sar9O5j0pnGLcXiJ0xiZEJogV42y3eKeNMpqdCmJLK59LDclUu4wBAS\r\n         fblhOecWTr2DMeqVEDvJCiQY2SjNDqlqO3N55lBkxweqqFTlaliE0bYcytq/+pswqpnS\r\n         PtCslt9+TbYGB+TrusSMV7fAZzKYD4NIJE3Wyu8c9PQ0jlcCkOsb9hBk5pJeHrozpFWt\r\n         3fK6Zqvc2sKVoXcOsOMSz7C+uRUaSl+XIbX8CoLQ0X7Q99HdadnTcv+sQUd3RxIkwVBi\r\n         t1iA==\r\nARC-Message-Signature: i=1; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=arc-20160816;\r\n        h=mime-version:subject:message-id:to:from:date:dkim-signature\r\n         :dkim-signature:arc

In [11]:
#This function cleans the boilerplate content of the fetched emails to return clean email bodies
def get_body():
    bodies=[]
    for mail in mails:
        paras=[]
        paragraphs = justext.justext(str(mail[0][1]), justext.get_stoplist("English"))
        for paragraph in paragraphs:
            if not paragraph.is_boilerplate:
                 paras.append(paragraph.text)
        text=''
        for para in paras:
            text=text+re.sub('\\\\n|\\\\r|=|--|<.*?>|div>|.*Content-Type:','',para)
        text_list=text.split(' ')
        text=''
        for t in text_list:
            if len(t)<20:
                text=text+' '+t
        bodies.append(text)
    return bodies

In [12]:
bodies=get_body()

In [13]:
#Writing cleaned emails to a txt file
with open('job_emails.txt','w+') as fp:
    for body in bodies:
        fp.write(body)
        fp.write('\n\t'+'-'*49+'\n')